In [1]:
import pandas as pd
import os
import sklearn
import numpy as np
from utils import split_dataset

In [2]:
covariates = pd.read_csv("./data/bdataaug.csv", index_col="Dates", skiprows=5)
covariates.index = pd.to_datetime(covariates.index)

labels = pd.read_csv("./data/jkse.csv", skiprows=0, index_col='Date')
labels.index = pd.to_datetime(labels.index)

labels = labels.rename(columns={"Close": "JKSE_PRICE"})
labels['PCT_CHANGE_20_JKSE'] = ((labels['JKSE_PRICE'].shift(-20) - labels['JKSE_PRICE']) / labels['JKSE_PRICE']) * 100

Split the dataset per ticker

In [3]:
# exclude the last 60 observations for evaluation
# covariates = covariates.iloc[:-40, :]
# ood_validation = covariates.iloc[-40:-20, :]

In [4]:
covlist = split_dataset(covariates)

## Create technical covariates


### PE Band & P/E Ratio  
$$
 \text{P/E Ratio} = \frac{1}{\text{EARN\_YLD}} 
$$

* $PE Band_q $ is the q-th percentile of P/E Ratio in a 60 trading days rolling window

### Mean Average Convergence/Divergence (MACD)

- MACD is the difference of short term $\text{EMA}_{12}$ and long term $\text{EMA}_{26}$ (signal for momentum)
- MACD Signal is the signal line of 9 day EMA of the MACD
- MACD Histogram is the difference between the MACD line and the signal line

In [5]:
colnames = covlist[0].columns.tolist()
for cov in covlist:
    # unify column names
    cov.columns = colnames
for cov in covlist:
    # Volume = Turnover / Close Price
    cov['VOLUME'] = cov['TURNOVER'] / cov['PX_LAST']
    # Calculate percent change * 100
    cov['PCT_CHANGE_20'] = ((cov['PX_LAST'].shift(-20) - cov['PX_LAST']) / cov['PX_LAST']) * 100
    # Ratio 10/30 = mean volume ratio for the last 10 days / mean volume ratio for the last 30 days
    cov['VOL_RATIO_10_20'] = cov['VOLUME'].rolling(window=10).mean() / cov['VOLUME'].rolling(window=20).mean()
    cov['VOL_RATIO_20_40'] = cov['VOLUME'].rolling(window=20).mean() / cov['VOLUME'].rolling(window=40).mean()
    cov['VOL_RATIO_40_80'] = cov['VOLUME'].rolling(window=40).mean() / cov['VOLUME'].rolling(window=80).mean()
    cov['VOL_RATIO_80_120'] = cov['VOLUME'].rolling(window=80).mean() / cov['VOLUME'].rolling(window=120).mean()
    
    # PE Band
    cov['PE_Ratio'] = 1 / cov['EARN_YLD']
    win = 60  # Set the rolling window period
    cov['PE_Band_25'] = cov['PE_Ratio'].rolling(win).quantile(0.25)
    cov['PE_Band_50'] = cov['PE_Ratio'].rolling(win).quantile(0.50)
    cov['PE_Band_75'] = cov['PE_Ratio'].rolling(win).quantile(0.75)

    # Calculate the 12-day EMA of PX_LAST
    ema_12 = cov['PX_LAST'].ewm(span=12, adjust=False).mean()

    # Calculate the 26-day EMA of PX_LAST
    ema_26 = cov['PX_LAST'].ewm(span=26, adjust=False).mean()

    # Calculate MACD
    cov['MACD'] = ema_12 - ema_26

    # Calculate the Signal line (9-day EMA of MACD)
    cov['MACD_Signal'] = cov['MACD'].ewm(span=9, adjust=False).mean()

    # Optionally, you can also calculate the MACD Histogram (the difference between MACD and Signal line)
    cov['MACD_Histogram'] = cov['MACD'] - cov['MACD_Signal']
    
    # Example: Momentum Indicator for various lags
    lags = [10, 20, 30, 60, 120]
    for lag in lags:
        cov[f'MOMENTUM_{lag}'] = cov['PX_LAST'] / cov['PX_LAST'].shift(lag)
        cov[f'TURNOVER_{lag}'] = cov['TURNOVER'].rolling(window=lag).mean()
        cov[f'PX_MOMENTUM_{lag}'] = cov['PX_LAST'] / cov['PX_LAST'].shift(lag)
        cov[f'PX_REVERSAL_{lag}'] = cov['PX_LAST'].shift(lag) / cov['PX_LAST']
        cov[f'VOLATILITY_{lag}'] = cov['PX_LAST'].rolling(window=lag).std()
        cov[f'VOLUME_STD_{lag}'] = cov['VOLUME'].rolling(window=lag).std()

In [6]:
# Remove duplicate indices if any
labels = labels[~labels.index.duplicated(keep='first')]

for i in range(len(covlist)):
    cov = covlist[i]
    cov = cov[~cov.index.duplicated(keep='first')]

    # Explicitly create a copy of the cov DataFrame to avoid SettingWithCopyWarning
    cov_copy = cov.copy()

    # Align the DataFrames on their indices (dates)
    aligned_df = labels.join(cov_copy[['PCT_CHANGE_20']], how='inner')

    # Calculate the difference and store it in cov_copy DataFrame
    cov_copy.loc[aligned_df.index, 'DELTA_20_CHANGE'] = aligned_df['PCT_CHANGE_20'] - aligned_df['PCT_CHANGE_20_JKSE']

    # Update the original DataFrame in covlist
    covlist[i] = cov_copy


In [7]:
tickernames = [col[:4] for col in pd.read_csv("./data/bdataaug.csv", skiprows=3).columns if not col.startswith("Unnamed")]

/var/folders/5w/sk_z507j78j69yh904rh10440000gn/T/ipykernel_3562/3641406362.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,

In [8]:
for i in range(len(covlist)):
    covlist[i]['Ticker'] = tickernames[i]

In [9]:
def unique_index(df, suffix):
    df_copy = df.copy()
    new_index = [f"{date.strftime('%d-%B-%Y')}-{suffix}" for date in df.index]
    df_copy.index = new_index
    return df_copy

Create reindexed covlist + pool to find quantiles + insert index back to other covariates

In [11]:
covlist_reindex = []
for idx, cov in enumerate(covlist):
    cov['Ticker'] = cov['Ticker'].unique()[0]
    covlist_reindex.append(unique_index(cov, idx))
    
pooled_df = pd.DataFrame()
for i, df in enumerate(covlist_reindex):
    pooled_df = pd.concat([pooled_df, df['DELTA_20_CHANGE']])

pooled_df['DELTA_20_QUINTILES'] = pd.qcut(pooled_df[0], q=10, labels=range(1, 11))

for i, df in enumerate(covlist_reindex):
    df['DELTA_20_QUINTILES'] = pooled_df.loc[df.index, 'DELTA_20_QUINTILES']
    covlist_reindex[i] = df

In [13]:
for cov in covlist_reindex:
    cov['TOP_5'] = cov['DELTA_20_QUINTILES'].apply(lambda x: 1 if x in [8, 9, 10] else 0)

Drop NaNs

In [14]:
for i in range(len(covlist_reindex)):
    covlist_reindex[i] = covlist_reindex[i].dropna(axis=0, how='any')
    covlist_reindex[i] = covlist_reindex[i].reset_index(drop=False)

save data

In [15]:
def rolling_window_save(covlist_reindex, train_window=252, test_window=60, valid=False):
    tdf = []
    vdf = []

    for cov in covlist_reindex:
        # Remove specific columns and reset the index but keep the original one
        # cov = cov.drop(['DELTA_20_CHANGE'], axis=1) if valid else cov.drop(['DELTA_20_CHANGE', 'PCT_CHANGE_20'], axis=1)

        # Iterate over the DataFrame with a rolling window approach
        for start in range(0, len(cov) - train_window - test_window + 1, test_window):
            end_train = start + train_window
            end_test = end_train + test_window

            train_df = cov.iloc[start:end_train].reset_index(drop=False)
            test_df = cov.iloc[end_train:end_test].reset_index(drop=False)

            tdf.append(train_df)
            vdf.append(test_df)

    return tdf, vdf

# Example usage
tdf, vdf = rolling_window_save(covlist_reindex, 252, 60)

In [21]:
tdf.append(vdf[-1])
vdf.pop()

,level_0,index,EARN_YLD,PX_TO_BOOK_RATIO,PX_TO_SALES_RATIO,RETURN_ON_ASSET,RETURN_COM_EQY,CUR_MKT_CAP,TRAIL_12M_NET_INC_GROWTH,OPER_INC_GROWTH,...,MOMENTUM_120,TURNOVER_120,PX_MOMENTUM_120,PX_REVERSAL_120,VOLATILITY_120,VOLUME_STD_120,DELTA_20_CHANGE,Ticker,DELTA_20_QUINTILES,TOP_5
0,1932,22-February-2024-341,14.4958,0.5301,0.4466,1.1995,8.0953,2302050.450,130.1217,86.0555,...,0.773256,2.779375e+09,0.773256,1.293233,23.299289,2.995618e+07,10.758776,IMJS,9,1.0
1,1933,23-February-2024-341,14.3876,0.5341,0.4500,1.1995,8.0953,2319359.100,130.1217,86.0555,...,0.783626,2.842931e+09,0.783626,1.276119,23.165680,3.000598e+07,7.987877,IMJS,9,1.0
2,1934,26-February-2024-341,14.3876,0.5341,0.4500,1.1995,8.0953,2319359.100,130.1217,86.0555,...,0.797619,2.843159e+09,0.797619,1.253731,23.101869,3.000447e+07,4.116970,IMJS,8,1.0
3,1935,27-February-2024-341,14.7171,0.5221,0.4399,1.1995,8.0953,2267433.150,130.1217,86.0555,...,0.761628,2.847997e+09,0.761628,1.312977,22.976001,3.000035e+07,6.058882,IMJS,8,1.0
4,1936,28-February-2024-341,14.7171,0.5221,0.4399,1.1995,8.0953,2267433.150,130.1217,86.0555,...,0.766082,2.848590e+09,0.766082,1.305344,22.862094,2.999872e+07,7.793067,IMJS,9,1.0
5,1937,29-February-2024-341,15.0620,0.5102,0.4298,1.1995,8.0953,2215507.200,130.1217,86.0555,...,0.748538,2.878268e+09,0.748538,1.335938,22.812253,2.999410e+07,10.456544,IMJS,9,1.0
6,1938,01-March-2024-341,14.8303,0.5182,0.4365,1.1995,8.0953,2250124.500,130.1217,86.0555,...,0.778443,2.889131e+09,0.778443,1.284615,22.805664,2.998859e+07,9.676236,IMJS,9,1.0
7,1939,04-March-2024-341,15.0620,0.5102,0.4298,1.1995,8.0953,2215507.200,130.1217,86.0555,...,0.766467,2.888277e+09,0.766467,1.304688,22.837975,2.998868e+07,9.682144,IMJS,9,1.0
8,1940,05-March-2024-341,15.4235,0.4982,0.4198,1.1995,8.0953,2163581.250,130.1217,86.0555,...,0.726744,2.882451e+09,0.726744,1.376000,22.782856,2.999004e+07,9.056059,IMJS,9,1.0
9,1941,06-March-2024-341,15.5479,0.4942,0.4164,1.1995,8.0953,2146272.600,130.1217,86.0555,...,0.720930,2.875217e+09,0.720930,1.387097,22.733821,2.999413e+07,11.121966,IMJS,9,1.0


In [22]:
for v in vdf:
    v.index = v['index']
    v.drop(['level_0', 'index', 'DELTA_20_CHANGE'], axis=1, inplace=True)

In [23]:
for t in tdf:
    t.index = t['index']
    t.drop(['level_0', 'index', 'DELTA_20_CHANGE', 'PCT_CHANGE_20'], axis=1, inplace=True)

In [24]:
train = pd.concat(tdf, axis=0)
valid = pd.concat(vdf, axis=0)

In [25]:
x_valid = valid.drop(['PCT_CHANGE_20', 'DELTA_20_QUINTILES', 'Ticker', 'TOP_5'], axis=1)
y_valid = valid[['TOP_5']]

In [26]:
x_train = train.drop(['DELTA_20_QUINTILES', 'Ticker', 'TOP_5'], axis=1)
y_train = train[['TOP_5']]

In [27]:
import joblib

joblib.dump(x_valid, './models/yuge70_xvalid.joblib')
joblib.dump(y_valid, './models/yuge70_yvalid.joblib')
joblib.dump(valid, './models/yuge70_validfull.joblib')

['./models/yuge70_validfull.joblib']

In [28]:
x_valid.to_csv('./data/yuge70_xvalid.csv', index=True)
y_valid.to_csv('./data/yuge70_yvalid.csv', index=True)
x_train.to_csv('./data/yuge70_xtrain.csv', index=True)
y_train.to_csv('./data/yuge70_ytrain.csv', index=True)


In [ ]:
valid = pd.concat(, axis=0)
valid.index = valid['index']
valid.drop('index', axis=1, inplace=True)

In [ ]:
valid.loc[:, ['PX_LAST', "VOLATILITY_20", 'Ticker', 'DELTA_20_CHANGE', "DELTA_20_QUINTILES", "TOP_5", "PCT_CHANGE_20"]]

In [ ]:
folder_name = 'bdataaug'
os.makedirs(folder_name, exist_ok=True)
for i in range(len(tdf)):
    tdf[i].to_csv(f'{folder_name}/train_{i}.csv', index=False)
    vdf[i].to_csv(f'{folder_name}/valid_{i}_pred.csv', index=False)

In [ ]:
valid.to_csv(f'{folder_name}/valid_pred_all.csv', index=True)